0 导入包

In [1]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

1 加载数据

In [2]:
# csv -> dataframe
df = pd.read_csv('0ori_data/csv_data.csv', encoding='utf-8') # 操作数据用dataframe
df

,订单号,商品id,商品编码,下单时间,商品名称,数量,支付价格,成本价,采购成本价
0,2023083193163,6971973442200,6971973442200,2023-08-31 23:00:55,禧遇棉花糖,1,2.5,1.54,NaN
1,2023083141300,221055484,221055484,2023-08-31 22:59:39,好丽友薯愿小包装,1,4.3,2.25,NaN
2,2023083141300,6972946670095,6972946670095,2023-08-31 22:59:39,徐多谷北海道豆制品40g,1,1.2,0.70,NaN
3,2023083141300,6958621301096,6958621301096,2023-08-31 22:59:39,真之味鸡蛋干,2,2.4,1.42,NaN
4,2023083141300,6948004703494,6948004703494,2023-08-31 22:59:39,鲜引力甘栗仁小包装,1,5.5,3.70,NaN
...,...,...,...,...,...,...,...,...,...
9995,2023082772991,6927462202091,6927462202091,2023-08-27 11:57:25,金锣王特级火腿肠80g,1,2.5,1.60,NaN
9996,2023082780198,221055372,221055372,2023-08-27 11:57:11,卡尔顿鸡蛋松饼,2,5.9,3.74,NaN
9997,2023082753252,6920698400378,6920698400378,2023-08-27 11:56:47,康师傅红烧牛肉面（BIG桶）,1,6.0,4.64,NaN
9998,2023082753252,6920698433543,6920698433543,2023-08-27 11:56:47,康师傅酸辣牛肉面（BIG桶）,1,6.0,4.64,NaN


2 删除一些列

In [3]:
df = df.drop(columns=['订单号']) # 删除一列
df = df.drop(columns=['商品id']) # 删除一列
df = df.drop(columns=['商品编码']) # 删除一列
df = df.drop(columns=['下单时间']) # 删除一列
df = df.drop(columns=['成本价']) # 删除一列
df = df.drop(columns=['采购成本价']) # 删除一列
df

,商品名称,数量,支付价格
0,禧遇棉花糖,1,2.5
1,好丽友薯愿小包装,1,4.3
2,徐多谷北海道豆制品40g,1,1.2
3,真之味鸡蛋干,2,2.4
4,鲜引力甘栗仁小包装,1,5.5
...,...,...,...
9995,金锣王特级火腿肠80g,1,2.5
9996,卡尔顿鸡蛋松饼,2,5.9
9997,康师傅红烧牛肉面（BIG桶）,1,6.0
9998,康师傅酸辣牛肉面（BIG桶）,1,6.0


3 构造qa数据

In [4]:
# 处理逻辑示例
print(df.iloc[2]['商品名称'])
print(df.iloc[2]['数量'])
print(df.iloc[2]['支付价格'])

dict = {
    "input":f"购买商品名称：{df.iloc[2]['商品名称']}，购买数量：{df.iloc[2]['数量']}",
    "output":f"总价：{df.iloc[2]['支付价格']}"
}
dict

徐多谷北海道豆制品40g
1
1.2


{'input': '购买商品名称：徐多谷北海道豆制品40g，购买数量：1', 'output': '总价：1.2'}

In [5]:
# 批量处理数据
all_dict = []
for i in range(len(df)):
    dict = {
        "input":f"购买商品名称：{df.iloc[i]['商品名称']}，购买数量：{df.iloc[i]['数量']}",
        "output":f"总价：{df.iloc[i]['支付价格']}"
    }
    all_dict.append(dict)

# 转换成dataset格式
df = pd.DataFrame(all_dict)
dataset = Dataset.from_pandas(df) 
dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 10000
})

In [6]:
dataset[0]

{'input': '购买商品名称：禧遇棉花糖，购买数量：1', 'output': '总价：2.5'}

4 构造prompt格式数据

In [7]:
# 构造prompt数据
# 按照每一个模型自己的规则处理成prompt形式
# 训练时候规定了用户开头是什么词，模型开头是什么词。这些词叫做特殊词，会被大模型过滤掉。
# 这些prompt写法在其他官方教程和special_tokens_map这个文件中

# 1 构造一个函数
def create_prompt(x1,x2):
    # 定义整体格式
    d_prompt = '''<bos><start_of_turn>user
你现在是一名导购，你需要根据顾客给出的商品和数量告诉对方购买价格
{x_1}<end_of_turn>'
<start_of_turn>model
{x_2}<end_of_turn><eos>'''

    # 提示词批量替换
    prompt = d_prompt.format(
        x_1=x1,
        x_2=x2,
    )
    return prompt

# 2 循环处理每一行数据
prompt_list =[]
for i in range(len(dataset)):
    prompt_list.append(
        create_prompt(x1=dataset[i]['input'], x2=dataset[i]['output'])
    )
    # break # 循环1次试试

# 3 添加新的一列数据
dataset = dataset.add_column("prompt", prompt_list)
dataset

Dataset({
    features: ['input', 'output', 'prompt'],
    num_rows: 10000
})

In [8]:
dataset[0]

{'input': '购买商品名称：禧遇棉花糖，购买数量：1',
 'output': '总价：2.5',
 'prompt': "<bos><start_of_turn>user\n你现在是一名导购，你需要根据顾客给出的商品和数量告诉对方购买价格\n购买商品名称：禧遇棉花糖，购买数量：1<end_of_turn>'\n<start_of_turn>model\n总价：2.5<end_of_turn><eos>"}

5 构造token数据

In [9]:
# 加载你需要的tokenizer
model_path = "google/gemma-2b-it" # 模型名字或路径
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 把某一列数据tokne化
dataset = dataset.map(
    lambda x: # x相当于dataset[i] 进入一个遍历for循环 for x in dataset 然后return冒号后边的输出
    tokenizer(x["prompt"],padding='max_length',truncation=True), # 把dataset[i]["prompt"]token化
    remove_columns=['input', 'output'], # 删除哪些列
    batched=True
)
dataset

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 10000
})

6 切分数据集

In [10]:
# 切分为train，和test数据集
dataset = dataset.train_test_split(train_size=0.8, test_size=0.2, seed=42, shuffle=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

7 保存数据

In [11]:
dataset.save_to_disk('文字数据集')

Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]